[View in Colaboratory](https://colab.research.google.com/github/geotana/UdacityDL/blob/master/6_lstm_TB_big=cross_e.ipynb)

Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

18.5.15 検証結果が明らかにおかしい。
原因予想
h  r r a r r r a r r r a r r r a r r r a r r r 
1 学習データが非常に偏っている   => No

2 推論部分のロジックがおかしい

3 ab２文字にした、同じデータセットが2回回った => Yes Fixed

18.5.16 状況関わらず　

原因予想
・lossがマイナスなので、根本がおかしい。Graph周り。==> 推論のインプットとアウトプを突き合わせみる 問題なし！？

Average loss at step 1500: -775.086458 learning rate: 10.000000
Minibatch perplexity: 0.00
Validation set perplexity: 0.09
inp [' or what by joe peac', ' some places such as'] ans ['r what by joe peacot', 'ome places such as c'] pre ['rererererererererere', 'rererererererererere']
inp ['ott and writings by ', ' clearwater feature '] ans ['t and writings by fr', 'learwater feature re'] pre ['clclclclclclclclclcl', 'clclclclclclclclclcl']

全然学習しない。

18.6.21
学習しない原因は、ネットにあるかも
オリジナル：27文字を 27次元の1-Hot Vecで表現
中間層は64
10文字の並びで学習

bigram版：2文字合わせて786次元を128に圧縮
中間層は64 ==>中間層　128*2で試す

実質20文字の並びで学習

 ==>中間層　128*2で試す
 依然損失関数がマイナスで、増加していく。おかしい
 
 ==> unr 5で試す　１０文字の並びでの学習に相当
 変わる兆候なし

18.6.24
forumを読んで気がついたが、このネットワークは分類問題である。
しかし、出力が1ーHotVecになっていないので、損失関数としてクロスエントロピーを使うのが不適切である。
解決方法
1）出力が、N次元の点だとして、回帰問題のように、2点間の、2乗距離の和をとる
➡️学習が進まない。
unr = 10 * 2
num_nodes = 64*4
としても大差なし

2）出力を1-HotVecにして比較する


In [1]:
!ls drive/ColabData/
!dpkg -l  software-properties-common python-software-properties module-init-tools google-drive-ocamlfuse fuse

ls: cannot access 'drive/ColabData/': No such file or directory
Desired=Unknown/Install/Remove/Purge/Hold
| Status=Not/Inst/Conf-files/Unpacked/halF-conf/Half-inst/trig-aWait/Trig-pend
|/ Err?=(none)/Reinst-required (Status,Err: uppercase=bad)
||/ Name           Version      Architecture Description
+++-==============-============-============-=================================
un  software-prope <none>       <none>       (no description available)
dpkg-query: no packages found matching python-software-properties
dpkg-query: no packages found matching module-init-tools
dpkg-query: no packages found matching google-drive-ocamlfuse
dpkg-query: no packages found matching fuse


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools  2>&1 > /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse  2>&1 > /dev/null
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
from __future__ import print_function #@title デフォルトのタイトル テキスト
import os
import numpy as np
import random
import string
import zipfile
import pdb #pdb.set_trace()
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists("./drive/ColabData/"+filename):
    filename, _ = urlretrieve('http://mattmahoney.net/dc/' + filename, "./drive/ColabData/" + filename)
  else:
    filename = "./drive/ColabData/" + filename
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data

filename = maybe_download('text8.zip', 31344016)
text = read_data(filename)
print('Data size %d' % len(text))

def cutprint(ltext):
  print(len(ltext))
  print(ltext[0:10])
  print(ltext[10:20])
  print(ltext[20:50])

valid_size = 1000
valid_text = text[:valid_size]
cutprint(valid_text)

train_text = text[valid_size:]
cutprint(train_text)

Found and verified ./drive/ColabData/text8.zip
Data size 100000000
1000
 anarchism
 originate
d as a term of abuse first use
99999000
ons anarch
ists advoc
ate social relations based upo


Create a small validation set.

Utility functions to map characters to vocabulary IDs and back.

In [5]:
final_embeddings_np_voc_em = np.load('./drive/ColabData/final_embeddings.npy')
# voc -> vocabulary size
# em -> embeddings
print("final_embeddings.shape",final_embeddings_np_voc_em.shape)

embedding_size = 128 # Dimension of the embedding vector.
vocabulary_size0 = 728 # num of ab bc ...

import pickle
with open('./drive/ColabData/dictionary.pickle', mode='rb') as f:
  dictionary = pickle.load(f)

def char2ab_vec_np_em_1(char1,char2):  #2文字からnp_em_1 [ [0.1] [0.15]...  ]
  ab_seqnum = dictionary[ char1 + char2]
  ab_np_voc_1 = np.zeros((vocabulary_size0,1))  
  ab_np_voc_1[ab_seqnum][0] = 1.0   # 1-hot vector
  return (final_embeddings_np_voc_em.T @ ab_np_voc_1)

with open('./drive/ColabData/reverse_dictionary.pickle', mode='rb') as f:
  reverse_dictionary = pickle.load(f)
    
def em_vec2ab(np_em_1): #em vectorから文字列を返す
  argmax_1 = np.argmax( final_embeddings_np_voc_em @ np_em_1, axis = 0)
#   pdb.set_trace()
  return reverse_dictionary[argmax_1[0]]
  
print( em_vec2ab(char2ab_vec_np_em_1('q','u')), "|", em_vec2ab( char2ab_vec_np_em_1('i','s')) )


final_embeddings.shape (728, 128)
qu | is


Function to generate a training batch for the LSTM model.

In [6]:
batch_size= 64 #64
num_unrollings= 10 #10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size   # 1batch当たりのテキストサイズ 
    self._cursor_L64 = [ offset * segment for offset in range(batch_size)] #segmentのスタート位置
    self._last_batch = self._next_batch() #np (bs,em)
  
  def _next_batch(self): #ここは良い。順番にab cd 。。。と返している
    """Generate a single batch from the current cursor position in the data."""
    batch_np_bs_em = np.empty((0, embedding_size),dtype=np.float)

    for b in range(self._batch_size):
       a_pos = self._cursor_L64[b] % self._text_size #折り返し
       b_pos =  (a_pos + 1) % self._text_size #折り返し 
       batch_np_bs_em = np.append(batch_np_bs_em,
           char2ab_vec_np_em_1( self._text[a_pos], self._text[b_pos]) .T, axis=0)
#        pdb.set_trace()
       self._cursor_L64[b] =(b_pos  + 1) %  self._text_size #segment内の次のテキスト位置をセット abを考慮、折り返し あり

    return batch_np_bs_em # ab Word Vector
  
  def next(self): #あれ。ここも良さそう。
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch] #最初のab vecをnp_bs_emの先頭に LIST of np(bs,em)) 
    # batches[0] = ['on', 'of']  、batches[1] = ['s ', ' t'] 繋がっている
  
    for step in range(self._num_unrollings):
      batches.append(self._next_batch()) #後ろにunrolling分だけ追加
#     pdb.set_trace()  
    self._last_batch = batches[-1] #LIST末尾の要素をセット
    return batches #cursorから始まるunrolling+1個分のnp(bs,em)のLISTを返す
  
def np_bs_em2ab_li_bs(np_bs_em): # bs個のem vectorからbs個の文字列を返す

  ab_li_bs=[]
  for b in range(np_bs_em.shape[0]): 
    x_np_em = np_bs_em.T [ : , b]
    ab_li_bs.append(em_vec2ab( x_np_em[ : , np.newaxis]))
  return ab_li_bs
  
def characters(probabilities): #1-hot vecから1文字にして、bs個のリストを返す
#   Turn a 1-hot encoding or a probability distribution over the possible
#   characters back into its (most likely) character representation.　
  x = [id2char(c) for c in np.argmax(probabilities, 1)]
  return x
  
# def batches2string(batches): # np_bs_emのリストから文字列を返す　オリジナルなら [unr個] bs個のリスト
#   """Convert a sequence of batches back into their (most likely) string
#   representation."""
# #  fi-embed.T x ab_vec -> voc vect 最大の要素から reverse dict
#   string = []  #bs個の要素を持つ縦ベクトルのnum_unrollings個の要素を持つList
 
#   for step in range(len(batches)):
#     string.append( np_bs_em2ab_li_bs( batches[step] ))
#   return string

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0] # bs 個の空の要素を持ったリストを作る
  for b in batches: #unr個の繰り返し
    s = [''.join(x) for x in zip(s, np_bs_em2ab_li_bs(b))]
#     s = [''.join(x) for x in zip(s, characters(b))]
  return s

#next()の呼び出しで次々に教師データのバッチサイズ分の文字emベクトルを返すようなオブジェクトを作る
train_batches = BatchGenerator(train_text, batch_size, num_unrollings) 

#正しく、nextで訓練データが取得できるか確かめる #精査すべし！！
print("1:", batches2string( train_batches.next() ) ) # train_batches.next()はunrolling+1個分のnp(bs,em)のLIST
print("2:", batches2string( train_batches.next() ) ) # これをunrolling+1個 x bs個の文字で返す
# 1: [['on', 'n ', 'gn', ' d', 'of', 'at', 'st', 'ck'], ['s ', 'fr', 'if', 'ru', ' t', ' l', ' d', 'y '], ['an', 'om', 'ic', 'gs', 'he', 'ea', 'ai', 'ri'], ['ar', ' t', 'an', ' c', ' o', 'st', 'ly', 'ca'], ['ch', 'he', 't ', 'on', 'ri', ' n', ' c', 'rd'], ['is', ' n', 'th', 'fu', 'gi', 'ot', 'ol', 'o '], ['ts', 'at', 'an', 'si', 'na', ' p', 'le', 'th'], [' a', 'io', ' i', 'on', 'l ', 'ar', 'ge', 'is'], ['dv', 'na', 'n ', ' i', 'do', 'li', ' n', ' c'], ['oc', 'l ', 'je', 'na', 'cu', 'am', 'ew', 'la'], ['at', 'me', 'rs', 'bi', 'me', 'en', 'sp', 'ss']]
# 2: [['at', 'me', 'rs', 'bi', 'me', 'en', 'sp', 'ss'], ['e ', 'di', 'ey', 'li', 'nt', 't ', 'ap', 'ic'], ['so', 'a ', ' a', 'ty', ' f', 's ', 'er', ' i'], ['ci', 'an', 'nd', ' t', 'ax', 'op', ' i', 'nc'], ['al', 'd ', ' g', 'o ', ' m', 'po', 'n ', 'lu'], [' r', 'fr', 'ue', 'or', 'ac', 'si', 'th', 'de'], ['el', 'om', 'rn', 'ie', 'hi', 'ti', 'e ', 's '], ['at', ' p', 'se', 'nt', 'ne', 'on', 'un', 'lu'], ['io', 're', 'y ', ' o', 's ', ' a', 'it', 'cy'], ['ns', 'si', 'ha', 'ne', 'wi', ' s', 'ed', ' w'], [' b', 'de', 's ', 'se', 'th', 'ub', ' s', 'in']]
  
# ons anarch
# ists advoc
# ate social

valid_batches = BatchGenerator(valid_text, 1, 1)
print("3:", batches2string( valid_batches.next() ) )
print("4:", batches2string( valid_batches.next() ) )
# 3: [[' a'], ['na']]
# 4: [['na'], ['rc']]

# train:  ons anarchists advocate social relations based upon voluntary as
# valid:   anarchism originated as a term of abuse first used against earl

# オリジナルの実行 bs=8
# ['ons anarchi', 'n from the ', 'gnificant t', ' drugs conf', 'of the orig', 'at least no', 'st daily co', 'cky ricardo']
# ['ists advoca', ' national m', 'than in jer', 'fusion inab', 'ginal docum', 'ot parliame', 'ollege news', 'o this clas']
# [' a']
# ['an']

1: ['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' dimens

Simple LSTM Model.

In [10]:
vocabulary_size = embedding_size #1-hotの代わりab_em_vectorを使う

num_nodes = 64 #64
import tensorflow as tf
embedding_size = 128 # Dimension of the embedding vector.

graph = tf.Graph()
with graph.as_default():

  big_x = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1), name="big_x")
  big_m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1), name="big_m")
  big_b = tf.Variable(tf.zeros([1, num_nodes * 4]), name="big_b")
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False,name="saved_output")
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False,name="saved_state")
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size0], -0.1, 0.1),name="w")
  b = tf.Variable(tf.zeros([vocabulary_size0]),name="b")
  
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state): #receiving input, hidden(t-1) and state(t-1)
    
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    with tf.name_scope("lstm_cell"):
  
      split0, split1, split2, split3 = tf.split(tf.matmul(i, big_x) + tf.matmul(o, big_m) + big_b, num_or_size_splits=4, axis=1)
      input_gate = tf.sigmoid(split0)
      forget_gate = tf.sigmoid(split1)
      update = split2
      state = forget_gate * state + input_gate * tf.tanh(update) # state(t-1) --> state(t)
      output_gate = tf.sigmoid(split3)
    return output_gate * tf.tanh(state), state #returning output(t) and state(t)
 
  # Input data.

  def tf_bs_em2tf_bs_voc(tf_bs_em): #em vectorからvoc-vecを返す
    bs_flat = tf.argmax( tf.matmul( tf_bs_em, final_embeddings_np_voc_em.T ), 1 )
    return tf.one_hot( bs_flat, depth=vocabulary_size0 )
  
  train_data = list() #num_unrollings の展開はListで
  
  with tf.name_scope("train_data_in"):
    for _ in range(num_unrollings + 1):
      train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
 
  train_inputs = train_data[:num_unrollings]
 
  train_labels = list()
  with tf.name_scope("train_data_out"):
    for i in range(1, num_unrollings + 1):
      train_labels.append( tf_bs_em2tf_bs_voc( train_data[i] ) )
  
  # num_unrollings = 3  ->  a, b, c, d 
  # train_inputs = a, b, c 　  0番目から2番目
  # train_labels = b, c, d　  　1番目から3番目(最後)まで
  print("train_inputs : ", train_inputs)
  print("train_labels : ", train_labels)
  
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output #????初期値は
  state = saved_state
  
  with tf.name_scope("train_inputs"):
    for i in train_inputs: # a, b, c をインプットとして、アウトプットを全部計算する
      output, state = lstm_cell(i, output, state)
      outputs.append(output)

  # State saving across unrollings. logitsとlossの前にsaveをする
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b) # アウトプットから一遍に推論するnp[bs x em]を unroll個縦に 繰り返し #これでクロスエントロピー求めて良い？
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits), name="loss") #推論と正答の損失関数を計算する
#     loss = tf.reduce_mean( tf.square(tf.concat(train_labels, 0)- logits), name="loss") #回帰のようなやり方
    print("train_labels : ",train_labels)
    print("labels=tf.concat(train_labels, 0) : ", tf.concat(train_labels, 0))
    print("logits : ", logits)
    print("loss : ", loss)

  # Optimizer.
  global_step = tf.Variable(0, name="global_step")
  with tf.name_scope("Optimize"):  #意味
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate) #learning_rateを入力として、optimizerを初期化する
    gradients, v = zip(*optimizer.compute_gradients(loss)) #損失関数を入力として勾配とその変数を返す
    print("v: ",v) #どの変数について最適化されたか
    print("global_step: ",global_step) #どうやってインクリメントされるのか？
    #pdb.set_trace()
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25) #勾配の長さを1.25に調整
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits,name="Predictions")
  print("train_prediction : ",train_prediction)
  
  # Sampling and validation eval: batch 1, no unrolling.
  with tf.name_scope("Optimize"):    
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size]) #input
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    # called from 10 timer veri
    reset_sample_state = tf.group( 
        saved_sample_output.assign(tf.zeros([1, num_nodes])), 
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    
    sample_output, sample_state = lstm_cell( sample_input, saved_sample_output, saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
      sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b)) #called from 10 timer
    print("sample_prediction : ",sample_prediction)
    
  #TensorBoard
  tf.summary.scalar('Loss', loss) # yをスカラーとして記録
  merged = tf.summary.merge_all() # おまじない
  logdir = "./drive/ColabData/logdir"
#   if tf.gfile.Exists(logdir):
#     tf.gfile.DeleteRecursively(logdir) # ./logdirが存在する場合削除
  writer = tf.summary.FileWriter(logdir, graph) # 保存先を./logdirに設定

train_inputs :  [<tf.Tensor 'train_data_in/Placeholder:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_1:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_2:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_3:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_4:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_5:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_6:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_7:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_8:0' shape=(64, 128) dtype=float32>, <tf.Tensor 'train_data_in/Placeholder_9:0' shape=(64, 128) dtype=float32>]
train_labels :  [<tf.Tensor 'train_data_out/one_hot:0' shape=(64, 728) dtype=float32>, <tf.Tensor 'train_data_out/one_hot_1:0' shape=(64, 728) dtype=float32>, <tf.Tensor 'train_data_out/one_hot_2:0' shape=(64, 728) dtype=float32>, <tf.

In [0]:
def np_bs_em2np_bs_voc1hot(np_bs_em): #em vectorからvoc-vecを返す
  bs_flat = np.argmax( np.matmul( np_bs_em, final_embeddings_np_voc_em.T ), 1 )
  return np.eye( vocabulary_size0 )[bs_flat]

def logprob(predictions, labels):
#   Log-probability of the true labels in a predicted batch.
#   predictions ｂｓ個の出力 emベクトル、softmaxで確率に
#   labels ?
  predictions[predictions < 1e-10] = 1e-10
#   pdb.set_trace()
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
#   prediction[0] は　長さ 1のランダムなvoc要素数の１次元配列
#   Sample one element from a distribution assumed to be an array of normalized  probabilities.
#   結果的に0 から　voc-1 の整数をランダムに生成
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction): #ランダムな文字を　 1-hot で返す　=> abベクトル
  # Turn a (column) prediction into 1-hot encoded samples.
  # prediction ?
  p = np.zeros(shape=[1, vocabulary_size0], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  # 長さ 1のランダムなvoc要素数の横ベクトル 
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size0])
#   c =  b/np.sum(b, 1)[:, None]
  c =  b/np.sum(b, 1)
#   pdb.set_trace()
  return c

In [9]:
num_steps = 7001 #7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  
  for step in range(num_steps):
    
    batches = train_batches.next() ### ＜train_batches＞ の利用先
    
    feed_dict = dict()
    for i in range(num_unrollings + 1): 
      feed_dict[train_data[i]] = batches[i]  #何を入れて学習させたか　追ってみる
      
    summary, _, l, predictions, lr = session.run( [merged, optimizer, loss, 
                         train_prediction, learning_rate], feed_dict=feed_dict)
    
    #predictions = np 80,128   80 = bs x unr 　を np(bs, em)に分解 
    pred_batches = list()  #   bs x em のリスト
    for unr in range(num_unrollings) : 
      pred_batches.append(predictions[0 + unr : batch_size + unr,:])

    mean_loss +=  l
    #print("mean_loss", mean_loss, "l", l,"feed_dict",feed_dict,"predictions",predictions)
    
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      
#       labels = np.concatenate(list(batches)[1:])  ## Lablesが1-hotになった影響を考慮
      labels = list()
      for unr in range(1, num_unrollings + 1):
#         pdb.set_trace()
        labels.append( np_bs_em2np_bs_voc1hot(batches[unr]) )

      print('Minibatch perplexity: %.2f' % float( np.exp( logprob(predictions, np.concatenate( labels ))))) #ここも1-HOt
    
      writer.add_summary(summary, step) #TB
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        
        for _ in range(1):  #(5)
          
          char1 = 'j'
#           char1 = valid_text[random.randint(0, 999)]  #乱数やめてみる?
#           char2 = valid_text[random.randint(0, 999)]
          char2 = 't'
          
          feed =  char2ab_vec_np_em_1( char1, char2).T  #ab形式 2文字のem形式
          sentence = char1 + char2  #   先頭にセット
          #pdb.set_trace()
          reset_sample_state.run()  #ネット状態の初期化
          
          for _ in range(79):   #前の文字abから次の文字cdを予想、これを文にれんけつし、79回繰り返す
            prediction = sample_prediction.eval( {sample_input: feed} )
            sentence += np_bs_em2ab_li_bs(prediction)[0] #1-hotから文字にする
#             pdb.set_trace()
          print(sentence)
      
    #トレーニングデータを印刷
        print("inp",batches2string( batches[0:num_unrollings]), "ans",batches2string( batches[1:]), 
#         print("inp",batches2string( train_data[0:num_unrollings].eval), "ans",batches2string( batches[1:]), 
              "pre", batches2string(pred_batches) ) 
          
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.589996 learning rate: 10.000000
Minibatch perplexity: 727.78


ValueError: ignored

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---